Перевод статьи: https://www.machinelearningplus.com/nlp/training-custom-ner-model-in-spacy/

# Содержание
1. Введение в распознование именованных сущностей
2. Необходимость настройки NER
3. Обновление распознователя именованных сущностей
4. Формат выборки для обучения
5. Обучение NER модели
6. Предсказание модели на новом тексте
7. Как обучить NER с чистого листа в spaCy
8. Обучение новой сущности

## Введение в распознание именованных сущностей
**spaCy** - это библиотека в области NLP с открытым исходным кодом. Широко используемая из-за своей гибкости и продвинутых инструментов. Перед тем как углубляться непосредственно в задачу NER в spaCy, давйте быстро поймем чтоже такое задач **распознование именованных сущностей**.

**Named Entity Recognition** - это классическая задача NLP по нахождению обсуждаемых объектов в тексте.

**Модель NER** - это модель которая способна выполнять эту задачу. Она должна быть способна идентифицировать такие сущности как "Америка", "Эмили", "Лондон" и т.д., и маркировать их соответствующе `PERSON`, `LOCATION` и прочее. Этот очень полезный инструмент помогает в поиске информации. В spaCy NER осуществляется `pipeline` компонентом `ner`. Большинство моделей по умолчанию используют в себе `pipeline`.

In [22]:
# Load a spacy model and check if it has ner
import spacy
nlp = spacy.load("en_core_web_sm")

nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

В случае если ваша модель не имеет `ner`, вы можете добавить его используя метод `nlp.add_pipe()`

## 2. Необходимость настройки NER
Как вы видите, spaCy имеет встроенный pipeline `ner` для распознования имен. Хоть он довольно хорошо справляется со своей работой, но не всегда его точности может хватать для вашей задачи. Порой, слово обозначается как `PERSON` или `ORG` в зависимости от контекста. Также, иногда категории, которую вы хотите извлекать не предусмотрена в spaCy.

Давайте взглянем как стандратная модель NER справится со статье о компании в сфере цифровой торговли.

In [23]:
# Perfoming NER on E-commerce article

article_text = """India that previously comprised only a handful of players in the e-commerce space, is now home to many biggies and giants battling out with each other to reach the top. This is thanks to the overwhelming internet and smartphone penetration coupled with the ever-increasing digital adoption across the country. These new-age innovations not only gave emerging startups a unique platform to deliver seamless shopping experiences but also provided brick and mortar stores with a level-playing field to begin their online journeys without leaving their offline legacies.
In the wake of so many players coming together on one platform, the Indian e-commerce market is envisioned to reach USD 84 billion in 2021 from USD 24 billion in 2017. Further, with the rate at which internet penetration is increasing, we can expect more and more international retailers coming to India in addition to a large pool of new startups. This, in turn, will provide a major Philip to the organized retail market and boost its share from 12% in 2017 to 22-25% by 2021. 
Here’s a view to the e-commerce giants that are dominating India’s online shopping space:
Amazon – One of the uncontested global leaders, Amazon started its journey as a simple online bookstore that gradually expanded its reach to provide a large suite of diversified products including media, furniture, food, and electronics, among others. And now with the launch of Amazon Prime and Amazon Music Limited, it has taken customer experience to a godly level, which will remain undefeatable for a very long time. 

Flipkart – Founded in 2007, Flipkart is recognized as the national leader in the Indian e-commerce market. Just like Amazon, it started operating by selling books and then entered other categories such as electronics, fashion, and lifestyle, mobile phones, etc. And now that it has been acquired by Walmart, one of the largest leading platforms of e-commerce in the US, it has also raised its bar of customer offerings in all aspects and giving huge competition to Amazon. 

Snapdeal – Started as a daily deals platform in 2010, Snapdeal became a full-fledged online marketplace in 2011 comprising more than 3 lac sellers across India. The platform offers over 30 million products across 800+ diverse categories from over 125,000 regional, national, and international brands and retailers. The Indian e-commerce firm follows a robust strategy to stay at the forefront of innovation and deliver seamless customer offerings to its wide customer base. It has shown great potential for recovery in recent years despite losing Freecharge and Unicommerce. 

ShopClues – Another renowned name in the Indian e-commerce industry, ShopClues was founded in July 2011. It’s a Gurugram based company having a current valuation of INR 1.1 billion and is backed by prominent names including Nexus Venture Partners, Tiger Global, and Helion Ventures as its major investors. Presently, the platform comprises more than 5 lac sellers selling products in nine different categories such as computers, cameras, mobiles, etc. 

Paytm Mall – To compete with the existing e-commerce giants, Paytm, an online payment system has also launched its online marketplace – Paytm Mall, which offers a wide array of products ranging from men and women fashion to groceries and cosmetics, electronics and home products, and many more. The unique thing about this platform is that it serves as a medium for third parties to sell their products directly through the widely-known app – Paytm. 

Reliance Retail – Given Reliance Jio’s disruptive venture in the Indian telecom space along with a solid market presence of Reliance, it is no wonder that Reliance will soon be foraying into retail space. As of now, it has plans to build an e-commerce space that will be established on online-to-offline market program and aim to bring local merchants on board to help them boost their sales and compete with the existing industry leaders. 
Big Basket – India’s biggest online supermarket, Big Basket provides a wide variety of imported and gourmet products through two types of delivery services – express delivery and slotted delivery. It also offers pre-cut fruits along with a long list of beverages including fresh juices, cold drinks, hot teas, etc. Moreover, it not only provides farm-fresh products but also ensures that the farmer gets better prices. 

Grofers – One of the leading e-commerce players in the grocery segment, Grofers started its operations in 2013 and has reached overwhelming heights in the last 5 years. Its wide range of products includes atta, milk, oil, daily need products, vegetables, dairy products, juices, beverages, among others. With its growing reach across India, it has become one of the favorite supermarkets for Indian consumers who want to shop grocery items from the comforts of their homes. 

Digital Mall of Asia – Going live in 2020, Digital Mall of Asia is a very unique concept coined by the founders of Yokeasia Malls. It is designed to provide an immersive digital space equipped with multiple visual and sensory elements to sellers and shoppers. It will also give retailers exclusive rights to sell a particular product category or brand in their respective cities. What makes it unique is its zero-commission model enabling retailers to pay only a fixed amount of monthly rental instead of paying commissions. With its one-of-a-kind features, DMA is expected to bring
never-seen transformation to the current e-commerce ecosystem while addressing all the existing e-commerce worries such as counterfeiting."""

In [24]:
doc = nlp(article_text)
for ent in doc.ents:
    print(ent.text, ent.label_)

India GPE
one CARDINAL
Indian NORP
USD 84 billion MONEY
2021 DATE
USD 24 billion MONEY
2017 DATE
India GPE
Philip PERSON
12% PERCENT
2017 DATE
22-25% PERCENT
2021 DATE
India GPE
Amazon ORG
One CARDINAL
Amazon ORG
Amazon ORG
Amazon Music Limited ORG
2007 DATE
Flipkart ORG
Indian NORP
Amazon ORG
Walmart ORG
one CARDINAL
US GPE
Amazon ORG
2010 DATE
Snapdeal PERSON
2011 DATE
more than 3 CARDINAL
India GPE
over 30 million CARDINAL
800+ CARDINAL
over 125,000 CARDINAL
Indian NORP
recent years DATE
Freecharge ORG
Unicommerce ORG
Indian NORP
ShopClues ORG
July 2011 DATE
Gurugram ORG
INR ORG
1.1 billion CARDINAL
Nexus Venture Partners ORG
Tiger Global PERSON
Helion Ventures ORG
nine CARDINAL
Paytm PERSON
Paytm Mall PERSON
third ORDINAL
Indian NORP
Reliance ORG
Reliance ORG
India GPE
Big Basket ORG
two CARDINAL
One CARDINAL
Grofers ORG
2013 DATE
the last 5 years DATE
atta ORG
daily DATE
India GPE
Indian NORP
2020 DATE
Digital Mall FAC
Asia LOC
Yokeasia Malls ORG
zero-commission MONEY
monthly DATE

Посмотрим на выход. Заметим, что FLIPKART был идентифицирован как `PERSON`, хотя должен быть `ORG`(на момент статьи наверное так было. Сейчас он обозначается как `ORG`). "Walmart" также был опознан неверно как `LOC`, а по контесту ему следует быть `ORG`. Есть другие примеры неверного распознования.

В таких случаях вы можете столкнуться с необходимостью улучшить и дообучить NER по необходимости конкретного контекста. Следующий раздел покажет вам как это сделать.

## 3. Обновление распознователя именованных сущностей
В прошлом разделе, вы могли увидеть почему нам нужно дообучить модель NER. Давайте продолжим и посмотрим как это сделать.

Давайте предположим, что вы распологаете разнообразными текстами о клиентских отзывах и компаниях. Наша задача убидиться что NER распознает компании, как `ORG`, а не как `PERSON`, помечает продукцию как `PRODUCT` и т.д.

Для этого вам нужно предоставить модели примеры на которых в будущем NER сможет учиться.

Чтобы это сделать, давайте использовать уже предобученную модель spaCy и улучшим ее новыми примерами.

Сначала, давайте загрузим уже существующую модель с компонентом `ner`. Затем, получим NER с помощью метода `get_pipe()`.

In [21]:
# Load pre-existing spacy model
import spacy
nlp = spacy.load("en_core_web_sm")

# Getting the pipeline component
ner = nlp.get_pipe("ner")

Чтобу улучшить предобученную модель новыми примерами, вы должны предоставить множество примеров, чтобы существенно улучшить систему - несколько сотен это хороший старт, но чем больше, тем лучше.

## 4. Формат выборки для обучения
spaCy принмиает тренировочные данные как список кортежей.

Каждый кортеж должен содердать текст и словарь. Словарь должен содержать началльные и конечные индексы именованной сущности в тексте, и категорию обозначаемой сущности.

Например: `("Walmart is a leading e-commerce company", {"entities": [(0,7, "ORG")]})`

In [7]:
# training data
TRAIN_DATA = [
              ("Walmart is a leading e-commerce company", {"entities": [(0, 7, "ORG")]}),
              ("I reached Chennai yesterday.", {"entities": [(19, 28, "GPE")]}),
              ("I recently ordered a book from Amazon", {"entities": [(24,32, "ORG")]}),
              ("I was driving a BMW", {"entities": [(16,19, "PRODUCT")]}),
              ("I ordered this from ShopClues", {"entities": [(20,29, "ORG")]}),
              ("Fridge can be ordered in Amazon ", {"entities": [(0,6, "PRODUCT")]}),
              ("I bought a new Washer", {"entities": [(16,22, "PRODUCT")]}),
              ("I bought a old table", {"entities": [(16,21, "PRODUCT")]}),
              ("I bought a fancy dress", {"entities": [(18,23, "PRODUCT")]}),
              ("I rented a camera", {"entities": [(12,18, "PRODUCT")]}),
              ("I rented a tent for our trip", {"entities": [(12,16, "PRODUCT")]}),
              ("I rented a screwdriver from our neighbour", {"entities": [(12,22, "PRODUCT")]}),
              ("I repaired my computer", {"entities": [(15,23, "PRODUCT")]}),
              ("I got my clock fixed", {"entities": [(16,21, "PRODUCT")]}),
              ("I got my truck fixed", {"entities": [(16,21, "PRODUCT")]}),
              ("Flipkart started it's journey from zero", {"entities": [(0,8, "ORG")]}),
              ("I recently ordered from Max", {"entities": [(24,27, "ORG")]}),
              ("Flipkart is recognized as leader in market",{"entities": [(0,8, "ORG")]}),
              ("I recently ordered from Swiggy", {"entities": [(24,29, "ORG")]})
              ]

Код выше показывает вам чистый формат тренировчных данных. Вам нужно добавить эти лейблы к использованию ner через `ner.add_label()` pipeline. Код ниже демонстрирует это.

In [8]:
# Adding labels to the `ner`

for _, annotations in TRAIN_DATA:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

Теперь время обучать NER с помощью этих примеров. Но перед обучением, помните, что модель крмое `ner` имеет другие компоненты pipeline. Эти компоненты не должны пострадать при обучении.

Перед обученем следует отключить другие компоненты pipeline с помощью метода `nlp.disable_pipes()`

In [9]:
# Disable pipeline components you don't need to change
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

## 5. Обучение модели NER
Для начала, давайте поймем идеи внедрение перед кодом.
1. Чтобы обучить `ner` модель должна быть зациклена достаточным количеством итераций. Если в вашем обучении 5 или 6 итераций, это может быть недостаточно эффективно.
2. Перед каждой итерацией хорошей практикой будет перемешивание примеров, используя `random.shuffle()`. Это обеспечит что модель не будет обращать внимание на порядок примеров в своем обучении.
3. Данные для обучения обычно передаются группами.

Вы можете вызвать функцию `minibatch()`, которая вернет вам данные укомплектованные в пакетах. Функция принимает параметр `size`, который указывает на размер пакета. Вы можете использовать функцию `compounding` для генерации бесконечного ряда составляющих значений.(???). Функция `compounding()` принимает три значения на входе `start`(первое целочисленное значение), `stop` (максимально генерируемое значение) и `compound`. Это значение хранимое в `compound` является компаундирующим фактором для ряда. Если вы не понимаете о чем речь, зацените [эту](https://spacy.io/api/top-level#util) статью.

Для каждой итерации, модель `ner` обновляется с помощью `nlp.update()`. Параметры для `nlp.update()`:
- `docs`: ожидает количество пакетов текста на вход. Вы можете пропустить каждый пакет используя метод `zip`, который вернет вам пакеты текста и аннотаций.
- `golds`: вы можете передать, которые пройдут через zip-метод внутри функции.
- `drop`: отражает процент отсева.
- `losses`: словарь для удержания потерь по кажому компоненту pipeline. Создайте пустой словарь и передайте его.

Для каждого слова `update()` создает предсказание. Затем он проверяет аннотации и сверяется с ними. Если предсказание неверно, веса регулируются, для большей точности в следующей попытке.

Наконец, все обучение проводится в рамках модель NLP с отключенным pipeline, чтобы предотвратить участие других компонентов.

**Оригинальный пример работает для старой версии spaCy**

In [10]:
# Import requirements
import random
from spacy.util import minibatch, compounding
from pathlib import Path

# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):

    # Training for 30 iterations
    for iterations in range(30):

        # shuffling examples before every iteration
        random.shuffle(TRAIN_DATA)
        losses = {}
        # batch up the examples using spaCy's minibatch
        batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(
                texts, # batch of texts
                annotations, # batch of annotations
                drop = 0.5, # dropout - make it harder to memorise data
                losses=losses,
            )
            print(f"Losses: {losses}")

ValueError: [E989] `nlp.update()` was called with two positional arguments. This may be due to a backwards-incompatible change to the format of the training data in spaCy 3.0 onwards. The 'update' function should now be called with a batch of Example objects, instead of `(text, annotation)` tuples. 

In [28]:
# Import requirements
import random
from pathlib import Path
from spacy.training.example import Example

# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):

    # Training for 30 iterations
    for iterations in range(30):

        # shuffling examples before every iteration
        random.shuffle(TRAIN_DATA)
        losses = {}

        #batch up the examples using spaCy's minibatch
        for batch in spacy.util.minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001)):
            for text, annotations in batch:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                # Update the model
                nlp.update(
                    [example],
                    losses=losses,
                    drop=0.5
            )
            print(f"Losses: {losses}")

c:\Users\muzzo\.virtualenvs\НИР_2-WuA8f2ou\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "I reached Chennai yesterday." with entities "[(19, 28, 'GPE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\muzzo\.virtualenvs\НИР_2-WuA8f2ou\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "I rented a screwdriver from our neighbour" with entities "[(12, 22, 'PRODUCT')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\muzzo\.virtualenvs\НИР_2-WuA8f2ou\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "I bought a new Washer

Losses: {'ner': 1.2440795134698972}


c:\Users\muzzo\.virtualenvs\НИР_2-WuA8f2ou\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "I rented a camera" with entities "[(12, 18, 'PRODUCT')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


Losses: {'ner': 1.244079758483601}


c:\Users\muzzo\.virtualenvs\НИР_2-WuA8f2ou\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "I bought a fancy dress" with entities "[(18, 23, 'PRODUCT')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\muzzo\.virtualenvs\НИР_2-WuA8f2ou\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "I got my clock fixed" with entities "[(16, 21, 'PRODUCT')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\muzzo\.virtualenvs\НИР_2-WuA8f2ou\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "I repaired my computer" with entities "[(15,

Losses: {'ner': 2.503857192607865}


c:\Users\muzzo\.virtualenvs\НИР_2-WuA8f2ou\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "I recently ordered from Swiggy" with entities "[(24, 29, 'ORG')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\muzzo\.virtualenvs\НИР_2-WuA8f2ou\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "I recently ordered a book from Amazon" with entities "[(24, 32, 'ORG')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\muzzo\.virtualenvs\НИР_2-WuA8f2ou\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "I got my truck fixed" with 

Losses: {'ner': 4.014475314118136}


c:\Users\muzzo\.virtualenvs\НИР_2-WuA8f2ou\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "I bought a old table" with entities "[(16, 21, 'PRODUCT')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\muzzo\.virtualenvs\НИР_2-WuA8f2ou\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "I rented a tent for our trip" with entities "[(12, 16, 'PRODUCT')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


Losses: {'ner': 4.014475987299401}
Losses: {'ner': 0.01703479488062577}
Losses: {'ner': 0.01703479488328802}
Losses: {'ner': 0.0170361193126194}
Losses: {'ner': 0.017036212491287308}
Losses: {'ner': 0.017036212625621945}
Losses: {'ner': 0.0001516827455703087}
Losses: {'ner': 0.00015168274702582872}
Losses: {'ner': 0.00015168275051086918}
Losses: {'ner': 0.023547553961130764}
Losses: {'ner': 0.023547576293629476}
Losses: {'ner': 1.9120081825983253e-08}
Losses: {'ner': 1.9272496372519103e-08}
Losses: {'ner': 1.9275001955665587e-08}
Losses: {'ner': 3.026829897980295e-08}
Losses: {'ner': 6.28261193716378e-08}
Losses: {'ner': 9.380849382321637e-07}
Losses: {'ner': 1.0732864770392192e-06}
Losses: {'ner': 0.00013742051183593897}
Losses: {'ner': 0.00013903865715629484}
Losses: {'ner': 0.00013903865715643422}
Losses: {'ner': 1.5702018525498085e-13}
Losses: {'ner': 2.071511476035037e-11}
Losses: {'ner': 3.304516132389817e-11}
Losses: {'ner': 3.5582249745350976e-11}
Losses: {'ner': 3.588875049773

## 6. Тестирование модели на новом тексте
Теперь обучение NER завершено. Вы можжете протестировать работу NER. Если вы не удовлетворены результами, добавьте больше примеров в датасет и попробуйте снова.

In [29]:
# Testing the model
doc = nlp("I was driving a Alto")
print(f"Entities: {[(ent.text, ent.label_) for ent in doc.ents]}")

Entities: []


**В статье у автора получилось извлечь "Alto" в категорию `PRODUCT`**

Модель не запоминает примеры обучения. Она должна извлечь из них веса и быть способна обобщить их для новых примеров.

Когда вы получите результаты модели, которые вас удовлетворят, сохраните новую модель.

После сохранения, вы можете загрузить модель из директории в любое время используя `spacy.load()`

In [33]:
# Save the model to directory
output_dir = Path('spacy_models')
nlp.to_disk(output_dir)
print("Saved model to ", output_dir)

#Load the saved model and predict
print("Loading from ", output_dir)
nlp_updated = spacy.load(output_dir)
doc = nlp_updated("Fridge can be ordered in FlipKart")
print(f"Entities {[(ent.text, ent.label_) for ent in doc.ents]}")

Saved model to  spacy_models
Loading from  spacy_models
Entities [('Fridge', 'PRODUCT'), ('FlipKart', 'ORG')]


## 7. Как обучить NER с нуля в spaCy
Если вы не хотите использовать пред-обученную модель, вы можете создать свою пустую модель с помощью `spacy.blank()`, указав идентификатор языка. Для создания пустой модели на основе английского вы должны передать "en".

После этого, основные шаги для обучения схожи. Ключевые моменты, которые стоит помнить:
1. Так как модель пуста, она не имеет никаких элементов pipeline. Вы должны добавить `ner` компонент с помощью `add_pipe()`.
2. Вам не нужно отключать другие компоненты pipeline соответственно.
3. Вам понадобится куда более объемный набор данных в таком случае.
4. Перед обучением настройте новую модель с помощью `nlp.begin_training()`

Код ниже показывает шаги по созданию новой модели.

In [35]:
# Train NER from a blank spacy model
import spacy

nlp = spacy.blank("en")
nlp.add_pipe('ner')
nlp.begin_training()

После это вы можете делать теже самые процедуры, что и с прошлой пред-обученной моделью.

In [36]:
# Training for 30 iterations
for iterations in range(30):

    # shuffling examples before every iteration
    random.shuffle(TRAIN_DATA)
    losses = {}

    #batch up the examples using spaCy's minibatch
    for batch in spacy.util.minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001)):
        for text, annotations in batch:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            # Update the model
            nlp.update(
                [example],
                losses=losses,
                drop=0.5
        )
        print(f"Losses: {losses}")

c:\Users\muzzo\.virtualenvs\НИР_2-WuA8f2ou\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "I repaired my computer" with entities "[(15, 23, 'PRODUCT')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\muzzo\.virtualenvs\НИР_2-WuA8f2ou\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "I rented a tent for our trip" with entities "[(12, 16, 'PRODUCT')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\muzzo\.virtualenvs\НИР_2-WuA8f2ou\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "I bought a new Washer" with entities

Losses: {'ner': 13.813345491886139}
Losses: {'ner': 24.91245061904192}


c:\Users\muzzo\.virtualenvs\НИР_2-WuA8f2ou\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "I bought a old table" with entities "[(16, 21, 'PRODUCT')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\muzzo\.virtualenvs\НИР_2-WuA8f2ou\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "I recently ordered a book from Amazon" with entities "[(24, 32, 'ORG')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\muzzo\.virtualenvs\НИР_2-WuA8f2ou\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "I rented a screwdriver from our n

Losses: {'ner': 37.98668548837304}
Losses: {'ner': 46.05652277544141}


c:\Users\muzzo\.virtualenvs\НИР_2-WuA8f2ou\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "I got my truck fixed" with entities "[(16, 21, 'PRODUCT')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\muzzo\.virtualenvs\НИР_2-WuA8f2ou\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "I bought a fancy dress" with entities "[(18, 23, 'PRODUCT')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


Losses: {'ner': 48.97552725486457}
Losses: {'ner': 3.8638777513697278}
Losses: {'ner': 5.431040444876999}
Losses: {'ner': 8.793471081529788}
Losses: {'ner': 12.456545059123897}
Losses: {'ner': 12.464889374675469}
Losses: {'ner': 1.9791981454558565}
Losses: {'ner': 5.951718816328161}
Losses: {'ner': 7.84543448425255}
Losses: {'ner': 9.226640014595853}
Losses: {'ner': 12.924371926896573}
Losses: {'ner': 3.2098916093455774}
Losses: {'ner': 5.2045615192908405}
Losses: {'ner': 8.083584892926336}
Losses: {'ner': 9.321083490139491}
Losses: {'ner': 11.296569504956622}
Losses: {'ner': 9.667202684220891e-05}
Losses: {'ner': 2.236825981134593}
Losses: {'ner': 5.293663625170484}
Losses: {'ner': 7.817601093971397}
Losses: {'ner': 9.743123472353533}
Losses: {'ner': 0.5370544454512549}
Losses: {'ner': 3.818505804082564}
Losses: {'ner': 5.252454564604454}
Losses: {'ner': 5.252569901513592}
Losses: {'ner': 7.534278158197678}
Losses: {'ner': 2.4968953206098723}
Losses: {'ner': 3.734485207152737}
Losses:

## 8. Обучение новой категории сущности в spaCy
В прошлых секциях, мы видели как `ner` категоризирует слова корректно. Но что, если вы хотите ввести категорию сущности, которой нет в текущей модели? 

Например, у вас есть много тестовых данных о еде, потребляемой в различных областях. И вы хотите, чтобы NER классифицировал все продукту питания по категории `FOOD`. Но такой категории не существует. Что делать?

spaCy очень гибок и позволяет вам добавить новый вид сущности и обучить модель. Эта функция чрезвычайно полезно и позволяет вам легко добавить новый тип для лучшего поиска информации. Эта секция расскажет, как осуществить это.

Для начала, загрущите предобученную модель с компонентом `ner`, используя `get_pipe()`.

In [37]:
# Import and load the spacy model
import spacy
nlp = spacy.load("en_core_web_sm")

# Getting the ner component
ner = nlp.get_pipe('ner')

Далее вложите новую категорию сущности в переменную `LABEL`. Теперь, как модель узнает какие объекты стоит классифицировать новой категорией? Вы обучите ее на новых примерах. Примеры должны научить определять сущности класса `FOOD`. Подготовим тренировочные данные.

Формат данных такой же как и раньше.

In [38]:
# New label to add
LABEL = "FOOD"

# Training examples in the required format
TRAIN_DATA =[ ("Pizza is a common fast food.", {"entities": [(0, 5, "FOOD")]}),
              ("Pasta is an italian recipe", {"entities": [(0, 5, "FOOD")]}),
              ("China's noodles are very famous", {"entities": [(8,14, "FOOD")]}),
              ("Shrimps are famous in China too", {"entities": [(0,7, "FOOD")]}),
              ("Lasagna is another classic of Italy", {"entities": [(0,7, "FOOD")]}),
              ("Sushi is extemely famous and expensive Japanese dish", {"entities": [(0,5, "FOOD")]}),
              ("Unagi is a famous seafood of Japan", {"entities": [(0,5, "FOOD")]}),
              ("Tempura , Soba are other famous dishes of Japan", {"entities": [(0,7, "FOOD")]}),
              ("Udon is a healthy type of noodles", {"entities": [(0,4, "ORG")]}),
              ("Chocolate soufflé is extremely famous french cuisine", {"entities": [(0,17, "FOOD")]}),
              ("Flamiche is french pastry", {"entities": [(0,8, "FOOD")]}),
              ("Burgers are the most commonly consumed fastfood", {"entities": [(0,7, "FOOD")]}),
              ("Burgers are the most commonly consumed fastfood", {"entities": [(0,7, "FOOD")]}),
              ("Frenchfries are considered too oily", {"entities": [(0,11, "FOOD")]})
           ]

Теперь набор данных готов, мы можем идти дальше и увидеть как эти примеры обучат компонент `ner`.

Помните, что модель еще не знает, что такое `FOOD`.

Так, что нашей первой задачей будет добавить лэйбл к `ner` с помощью `add_label()`. Далее вы можете использовать `resume_training()`

Также, когда обучение завершится другие компоненты pipeline будут изменены. Для избежения этого, используйте `disable_pipes()`, чтобы отключить другие компоненты.

In [39]:
# Add the new label to ner
ner.add_label(LABEL)

# Resume training
optimizer = nlp.resume_training()
move_names = list(ner.move_names)

# List of pipes you want to train
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]

# List of pipes which should remain unaffected in training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [51]:
with nlp.disable_pipes(*other_pipes):
    sizes = compounding(1.0, 3.0, 1.001)

    for int in range(30):
        random.shuffle(TRAIN_DATA)
        batches = minibatch(TRAIN_DATA, size=sizes)
        losses = {}
        for batch in spacy.util.minibatch(TRAIN_DATA, size=sizes):
            for text, annotations in batch:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                # Update the model
                nlp.update(
                    [example],
                    sgd = optimizer,
                    losses=losses,
                    drop=0.35
            )
            print(f"Losses: {losses}")

Losses: {'ner': 1.0264057287137955e-14}
Losses: {'ner': 5.2657296632916986e-11}
Losses: {'ner': 5.2694514987082524e-11}
Losses: {'ner': 5.269451505774786e-11}
Losses: {'ner': 1.2674101764767096e-09}
Losses: {'ner': 1.271050526296091e-09}
Losses: {'ner': 1.286170666763914e-09}
Losses: {'ner': 1.28617877503987e-09}
Losses: {'ner': 1.286542526533046e-09}
Losses: {'ner': 1.2865447433613305e-09}
Losses: {'ner': 1.2976106841284021e-09}
Losses: {'ner': 1.2976107057646817e-09}
Losses: {'ner': 1.297647801246566e-09}
Losses: {'ner': 1.3057176304633046e-09}
Losses: {'ner': 9.898128249257072e-18}
Losses: {'ner': 4.424195743635072e-12}
Losses: {'ner': 5.57821306033705e-12}
Losses: {'ner': 5.585800876052629e-12}
Losses: {'ner': 5.588155277556233e-12}
Losses: {'ner': 1.2107972516202681e-11}
Losses: {'ner': 1.2465177338294161e-11}
Losses: {'ner': 1.2498767274495398e-11}
Losses: {'ner': 3.1326966673672224e-11}
Losses: {'ner': 4.0633072958301925e-11}
Losses: {'ner': 4.0634787895673995e-11}
Losses: {'ner

In [52]:
# Tesing the NER
test_text = "I ate Sushi yesterday. Maggi is a common fast food "
doc = nlp(test_text)
print("Entities in '%s'" % test_text)
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities in 'I ate Sushi yesterday. Maggi is a common fast food '
Entities [('Maggi', 'FOOD')]
